<h1 align=center><font size = 5>Best Neibghorhood in Houston to Raise Kids </font></h1>

## Introduction

This notebook is created for Coursera Capstone project. Here we try to find the best Zip Code in Houston,TX to raise a family with school going kids. We would decide the best Zip Code on  4 parameters - Home Price, Crime Rate, School Rating and number of Venues in each neighborhood. We get neighbourhood and average home price data from website of Houstonia magazine ,Crimes data from website of Houston Police Department and School rating data from a downloaded CSV file from Texas Education Agency.  We will use Foursquare API to get the number of venues in each of neighborhood using its'latitude and longitude .

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Houston</a>

3. <a href="#item4">Cluster Neighborhoods</a>

4. <a href="#item5">Examine Clusters</a>    
</font>
</div>

First we will install the missing libaries and import them. 

In [8]:
!conda install -c conda-forge geopy --yes 

Solving environment: ...working... 
  - anaconda/win-64::conda-4.8.1-py37_0
  - defaults/win-64::conda-4.8.1-py37_0done

## Package Plan ##

  environment location: C:\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         182 KB  conda-forge
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       hfa6e2cd_0         4.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geo

In [ ]:
conda install -c anaconda beautifulsoup4

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
  - anaconda/win-64::conda-4.8.1-py37_0
  - defaults/win-64::conda-4.8.1-py37_0done

## Package Plan ##

  environment location: C:\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.0               |             py_0         606 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         704 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::a

In [1]:
conda install -c conda-forge geojson

Solving environment: ...working... 
  - anaconda/win-64::conda-4.8.1-py37_0
  - defaults/win-64::conda-4.8.1-py37_0done

## Package Plan ##

  environment location: C:\anaconda3

  added / updated specs:
    - geojson


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geojson-2.5.0              |             py_0          15 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          15 KB

The following NEW packages will be INSTALLED:

  geojson            conda-forge/noarch::geojson-2.5.0-py_0




geojson-2.5.0        | 15 KB     |            |   0% 
geojson-2.5.0        | 15 KB     | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

# import k-means from clustering stage
from sklearn.cluster import KMeans


## 1. Download and Explore Datasets


#### Get Home Prices data for each Zip Code in Houston

Scrape a page from website of Houstonia magzine to get the neibourhood data of Houston, TX using BeautifulSuop4 Library. This would return neighborhood name, Zip Code and averge home price in that Zip Code.

In [42]:
page = urlopen('https://www.houstoniamag.com/articles/2017/3/24/neighborhoods-by-the-numbers-real-estate-data-2017').read()
soup = bs(page)
soup.prettify()
table = soup.find('table')
df = pd.read_html(str(table))
nbrs= df[0] # get the first table
nbrs= nbrs[nbrs.columns[0:3]]
nbrs= nbrs.rename(columns={"Unnamed: 0": "Neighborhood", "ZIP Code": "Zip", "2016 Median Home Price":"HomePrice"})
nbrs.head()

,Neighborhood,Zip,HomePrice
0,1960/Cypress,77065,"$179,000"
1,Aldine Area,77039,"$133,500"
2,Alief,77072,"$164,000"
3,Alvin North,77511,"$227,000"
4,Alvin South,77511,"$163,900"


#### Get latitude and longitude data of each Zip Code in Houston

Next, we need to get the latitude and longitude data of each Zip Code. We would read this data from OpenDataSoft which is avilable in csv format at https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/Chicago&use_labels_for_header=true&csv_separator=%3B

In [43]:
zipdf  = pd.read_csv('https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/Chicago&use_labels_for_header=true&csv_separator=%3B', sep=';')
zipdf = zipdf[['Zip', 'Latitude', 'Longitude']]
zipdf["Zip"]= zipdf["Zip"].apply(str)
zipdf.head()

,Zip,Latitude,Longitude
0,71937,34.398483,-94.39398
1,72044,35.624351,-92.16056
2,56171,43.660847,-94.74357
3,49430,43.010337,-85.89754
4,52585,41.194129,-91.98027


#### Get Crime Data for each Zip Code in Houston

For crime data in each Zip Code, we would read the data from website of Houston Police Department where it is available in excel format.

In [44]:
crimedf = pd.read_excel('http://www.houstontx.gov/police/cs/xls/06-2019.NIBRS_Public_Data_Group_A&B.xlsx', header=11)
crimedf = crimedf[['ZIP', 'Offense Count']].groupby(['ZIP']).sum().reset_index()
crimedf = crimedf.rename(columns={ "ZIP": "Zip", "Offense Count":"Crimes"})
crimedf.head()

,Zip,Crimes
0,75248,1
1,77002,547
2,77003,183
3,77004,520
4,77005,91


#### Get Average School Rating of each Zip Code in Houston

Next, We would read the Texas Public School's accountability rating from webiste of ArcGis Site. This data seems to be from Texas Education Agency.

In [45]:
schooldf  = pd.read_csv('https://opendata.arcgis.com/datasets/6cf4436417ff43d0a6e741dc83339ae2_0.csv', sep=',')
schooldf = schooldf[['School_Nam', 'School_Str','School_Cit', 'School_Sta','School_Zip', 'Acc_Rating']]
schooldf = schooldf[(schooldf['School_Cit'] == 'HOUSTON') & (schooldf['School_Sta'] == 'TX')]
schooldf[['Zip','ZipExtn']]= schooldf['School_Zip'].str.split("-",expand=True) 
schooldf['Acc_Rating'] = schooldf[schooldf.columns[5]].replace('[\*,]', '', regex=True)
schooldf.dropna()
invalid_rating = ['Not Rated', 'NULL']
schooldf = schooldf[~schooldf.Acc_Rating.isin(invalid_rating)]
schooldf.head()

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (45,66,67,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,School_Nam,School_Str,School_Cit,School_Sta,School_Zip,Acc_Rating,Zip,ZipExtn
10,NORTHSIDE H S,1101 QUITMAN,HOUSTON,TX,77009-7815,C,77009,7815
12,ANDERSON ACADEMY,7401 WHEATLEY ST,HOUSTON,TX,77088-7845,F,77088,7845
21,FRAZIER EL,8300 LITTLE RIVER RD,HOUSTON,TX,77064-7904,C,77064,7904
45,YOUNG SCHOLARS ACADEMY FOR EXCELLENCE,1809 LOUISIANA,HOUSTON,TX,77002-8013,D,77002,8013
57,KETELSEN EL,600 QUITMAN,HOUSTON,TX,77009-8113,A,77009,8113


Next we clean the data, choose the schools for only Houston City with their accountability rating

We see that each school has rating between A to F. We will replace these ratings with 6 to 1. 6 for A, 5 for B and so on.
And then we take the average rating for each of Zip Code

In [46]:
schooldf.Acc_Rating[schooldf.Acc_Rating == 'A'] = 6
schooldf.Acc_Rating[schooldf.Acc_Rating == 'B'] = 5
schooldf.Acc_Rating[schooldf.Acc_Rating == 'C'] = 4
schooldf.Acc_Rating[schooldf.Acc_Rating == 'D'] = 3
schooldf.Acc_Rating[schooldf.Acc_Rating == 'E'] = 2
schooldf.Acc_Rating[schooldf.Acc_Rating == 'F'] = 1
schooldf['Acc_Rating'] = schooldf[schooldf.columns[5]].astype(float)
schooldf = schooldf.groupby(['Zip'], as_index=False).mean()
schooldf = schooldf.rename(columns={ "Acc_Rating": "Rating"})
schooldf.dropna()
schooldf.head()

,Zip,Rating
0,77002,4.500000
1,77003,5.500000
2,77004,4.181818
3,77005,6.000000
4,77006,4.000000


In [47]:
nbrs_merged = pd.merge(nbrs, zipdf, on='Zip', how='left')
nbrs_merged = pd.merge(nbrs_merged, crimedf, on='Zip', how='left')
nbrs_merged = pd.merge(nbrs_merged, schooldf, on='Zip', how='left')
nbrs_merged = nbrs_merged.dropna()
nbrs_merged.head()

,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating
1,Aldine Area,77039,"$133,500",29.909123,-95.33683,3.0,4.500000
2,Alief,77072,"$164,000",29.700898,-95.59002,426.0,4.181818
13,Braeswood Place,77025,"$715,000",29.690230,-95.43474,225.0,5.166667
14,Brays Oaks,77031,"$225,000",29.654132,-95.54311,146.0,5.000000
15,Briargrove,77057,"$824,000",29.745129,-95.49131,432.0,4.666667


In [48]:
nbrs_merged.count()

Neighborhood    62
Zip             62
HomePrice       62
Latitude        62
Longitude       62
Crimes          62
Rating          62
dtype: int64

Now we have all data - Home Price, Crime Data and Average School Rating for each Zip Code. Now we will merge all the data in single dataframe

## 2. Explore Neighborhoods in Houston

#### Use geopy library to get the latitude and longitude values of Houston City.

In [49]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="To_explorer")

from geopy.exc import GeocoderTimedOut
def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [50]:
 # convert an address into latitude and longitude values
address = 'HOUSTON,TX'
location = do_geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of HOUSTON,TX are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of HOUSTON,TX are 29.7589382, -95.3676974.


#### Use folium library to plot all neighbourhoods of  Houston City.

In [51]:
map_houston = folium.Map(location=[latitude, longitude], zoom_start=10)

map_houston


In [52]:
# add markers to map

for index, row in nbrs_merged.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    neighborhood = row['Neighborhood']
    label = neighborhood
    label = folium.Popup(label, parse_html=False)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston)

map_houston

#### Use Foursquare API to get the venues in Houston

In [75]:
CLIENT_ID = '' # Removed to publish on GitHub
CLIENT_SECRET = '' # Removed to publish on GitHub
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        response = requests.get(url, verify=False).json()["response"]
        print(response)
        results = response['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
houston_venues = getNearbyVenues(names=nbrs_merged['Neighborhood'],
                                   latitudes=nbrs_merged['Latitude'],
                                   longitudes=nbrs_merged['Longitude']
                                  )
houston_venues = houston_venues.groupby(['Neighborhood', 'Latitude', 'Longitude']).count()
houston_venues = houston_venues.drop(columns=['Venue Latitude','Venue Longitude','Venue Category'])
houston_venues = houston_venues.rename(columns={"Venue": "Venues"})
houston_venues

Aldine Area
{'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."}, 'headerLocation': 'Houston', 'headerFullLocation': 'Houston', 'headerLocationGranularity': 'city', 'totalResults': 0, 'suggestedBounds': {'ne': {'lat': 29.913623004500003, 'lng': -95.33164828277188}, 'sw': {'lat': 29.90462299549999, 'lng': -95.34201171722813}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': []}]}
Alief
{'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Alief', 'headerFullLocation': 'Alief, Houston', 'headerLocationGranularity': 'neighborhood', 'totalResults': 9, 'suggestedBounds': {'ne': {'lat': 29.705398004500005, 'lng': -95.58484905863216}, 'sw': {'lat': 29.696397995499993, 'lng': -95.59519094136783}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 

,,,Venues
Neighborhood,Latitude,Longitude,
Alief,29.700898,-95.59002,9
Braeswood Place,29.690230,-95.43474,5
Brays Oaks,29.654132,-95.54311,2
Briargrove,29.745129,-95.49131,4
Briargrove Park/Walnut Bend,29.741565,-95.55996,12
Briarmeadow/Tanglewilde,29.734379,-95.52269,23
Champions Area,29.984672,-95.52887,5
Charnwood/Briarbend,29.734379,-95.52269,23
Clear Lake Area,29.574930,-95.13238,2


#### Merge Home Price, Crime Data,  Average School Rating and Venues data for each Zip Code

In [57]:
nbrs_merged = pd.merge(nbrs, zipdf, on='Zip', how='left')
nbrs_merged = pd.merge(nbrs_merged, crimedf, on='Zip', how='left')
nbrs_merged = pd.merge(nbrs_merged, schooldf, on='Zip', how='left')
nbrs_merged = pd.merge(nbrs_merged, houston_venues, on='Neighborhood', how='left')
nbrs_merged = nbrs_merged.dropna()
nbrs_merged.head()

,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating,Venues
2,Alief,77072,"$164,000",29.700898,-95.59002,426.0,4.181818,9.0
13,Braeswood Place,77025,"$715,000",29.690230,-95.43474,225.0,5.166667,5.0
14,Brays Oaks,77031,"$225,000",29.654132,-95.54311,146.0,5.000000,2.0
15,Briargrove,77057,"$824,000",29.745129,-95.49131,432.0,4.666667,4.0
16,Briargrove Park/Walnut Bend,77042,"$460,000",29.741565,-95.55996,421.0,4.200000,12.0


In [58]:
nbrs_merged

,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating,Venues
2,Alief,77072,"$164,000",29.700898,-95.59002,426.0,4.181818,9.0
13,Braeswood Place,77025,"$715,000",29.690230,-95.43474,225.0,5.166667,5.0
14,Brays Oaks,77031,"$225,000",29.654132,-95.54311,146.0,5.000000,2.0
15,Briargrove,77057,"$824,000",29.745129,-95.49131,432.0,4.666667,4.0
16,Briargrove Park/Walnut Bend,77042,"$460,000",29.741565,-95.55996,421.0,4.200000,12.0
17,Briarmeadow/Tanglewilde,77063,"$292,000",29.734379,-95.52269,403.0,4.857143,23.0
21,Champions Area,77069,"$234,195",29.984672,-95.52887,7.0,5.000000,5.0
22,Charnwood/Briarbend,77063,"$527,500",29.734379,-95.52269,403.0,4.857143,23.0
23,Clear Lake Area,77062,"$233,000",29.574930,-95.13238,57.0,5.200000,2.0
30,Cottage Grove,77007,"$392,500",29.772627,-95.40319,419.0,6.000000,47.0


In [59]:
nbrs_final = nbrs_merged.copy()
nbrs_final.head()

,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating,Venues
2,Alief,77072,"$164,000",29.700898,-95.59002,426.0,4.181818,9.0
13,Braeswood Place,77025,"$715,000",29.690230,-95.43474,225.0,5.166667,5.0
14,Brays Oaks,77031,"$225,000",29.654132,-95.54311,146.0,5.000000,2.0
15,Briargrove,77057,"$824,000",29.745129,-95.49131,432.0,4.666667,4.0
16,Briargrove Park/Walnut Bend,77042,"$460,000",29.741565,-95.55996,421.0,4.200000,12.0


Now we normalize all data.

In [60]:
nbrs_final = nbrs_merged.copy()
nbrs_final['HomePrice'] = nbrs_final[nbrs_final.columns[2]].replace('[\$,]', '', regex=True).astype(float)
cols_to_norm = ['HomePrice','Crimes','Rating', 'Venues']
nbrs_final[cols_to_norm] = nbrs_final[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
nbrs_final.head()


,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating,Venues
2,Alief,77072,0.026221,29.700898,-95.59002,0.816764,0.480519,0.106667
13,Braeswood Place,77025,0.224138,29.690230,-95.43474,0.424951,0.761905,0.053333
14,Brays Oaks,77031,0.048132,29.654132,-95.54311,0.270955,0.714286,0.013333
15,Briargrove,77057,0.263290,29.745129,-95.49131,0.828460,0.619048,0.040000
16,Briargrove Park/Walnut Bend,77042,0.132543,29.741565,-95.55996,0.807018,0.485714,0.146667


Now we have all data for each Zip Code. We will use those data to cluster the Neighborhoods.

## 4. Cluster Neighborhoods in Houston


#### Run *k*-means to cluster the neighborhood into 5 clusters.

In [69]:
# set number of clusters
kclusters = 5

clustering_input = nbrs_final.drop(['Neighborhood','Zip', 'Latitude','Longitude'],  1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering_input)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 1, 1, 2, 2, 2, 1, 2, 1, 4, 2, 1, 1, 2, 1, 1, 2, 1, 2, 0, 0, 0,
       0, 1, 1, 2, 3, 4, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 4, 1, 3, 3,
       2, 1, 2, 3, 1, 4, 4, 1, 2, 2, 1])

#### create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [70]:
# add clustering labels
nbrs_final_clustered = nbrs_final.copy()
nbrs_final_clustered.insert(0, 'Cluster Labels', kmeans.labels_)
nbrs_final_clustered.head() 

,Cluster Labels,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating,Venues
2,2,Alief,77072,0.026221,29.700898,-95.59002,0.816764,0.480519,0.106667
13,1,Braeswood Place,77025,0.224138,29.690230,-95.43474,0.424951,0.761905,0.053333
14,1,Brays Oaks,77031,0.048132,29.654132,-95.54311,0.270955,0.714286,0.013333
15,2,Briargrove,77057,0.263290,29.745129,-95.49131,0.828460,0.619048,0.040000
16,2,Briargrove Park/Walnut Bend,77042,0.132543,29.741565,-95.55996,0.807018,0.485714,0.146667


####  Visualize the resulting clusters

In [71]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nbrs_final_clustered['Latitude'], nbrs_final_clustered['Longitude'], nbrs_final_clustered['Neighborhood'], nbrs_final_clustered['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [72]:
nbrs_final_clustered.loc[nbrs_final_clustered['Cluster Labels'] == 0, nbrs_final_clustered.columns[[1] + list(range(2, nbrs_final_clustered.shape[1]))]]

,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating,Venues
60,Humble Area East,77338,0.025144,30.005691,-95.28488,0.033138,0.0,0.426667
61,Humble Area South,77338,0.006214,30.005691,-95.28488,0.033138,0.0,0.426667
62,Humble Area West,77338,0.021552,30.005691,-95.28488,0.033138,0.0,0.426667
70,Kingwood South,77338,0.053520,30.005691,-95.28488,0.033138,0.0,0.426667



Neighborhoods in this cluster have low home prices and low crime rates. Public school ratings are not available. So  we are unable to recommend them.

#### Cluster 2

In [73]:
nbrs_final_clustered.loc[nbrs_final_clustered['Cluster Labels'] == 1, nbrs_final_clustered.columns[[1] + list(range(2, nbrs_final_clustered.shape[1]))]]

,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating,Venues
13,Braeswood Place,77025,0.224138,29.690230,-95.43474,0.424951,0.761905,0.053333
14,Brays Oaks,77031,0.048132,29.654132,-95.54311,0.270955,0.714286,0.013333
21,Champions Area,77069,0.051435,29.984672,-95.52887,0.000000,0.714286,0.053333
23,Clear Lake Area,77062,0.051006,29.574930,-95.13238,0.097466,0.771429,0.013333
40,East End Revitalized,77003,0.058190,29.749278,-95.34741,0.343080,0.857143,0.080000
41,Eldridge North,77041,0.043642,29.858730,-95.57243,0.120858,0.785714,0.053333
49,Galleria,77056,0.209770,29.747328,-95.46931,0.637427,0.809524,0.186667
50,Garden Oaks,77015,0.127155,29.778526,-95.18118,0.294347,0.688312,0.013333
53,Heights/Greater Heights,77008,0.132507,29.798777,-95.40951,0.481481,0.714286,0.226667
72,Knollwood/Woodside Area,77025,0.121767,29.690230,-95.43474,0.424951,0.761905,0.053333


When we examine cluster 2, we find that in this cluster, the home price is on lower side except Memorial Villages, Crime rate is lower except Sharpstown area and Number of Venues are also lower.
Some of the neighborhoods seems interesting -
1. Medical Center Area
2. Memorial Villages
3. West University
4. Rice/Museum District

These neighborhoods have higher school ratings, lower crimes but have lower venues for other activities. Further Medical center area has lower home prices while others have higher home prices.

We would recommend Zip Code - 77030 and 77005 for the families who are okay with little less venues for activities in their own neighborhood.


#### Cluster 3

In [74]:
nbrs_final_clustered.loc[nbrs_final_clustered['Cluster Labels'] == 2, nbrs_final_clustered.columns[[1] + list(range(2, nbrs_final_clustered.shape[1]))]]

,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating,Venues
2,Alief,77072,0.026221,29.700898,-95.59002,0.816764,0.480519,0.106667
15,Briargrove,77057,0.263290,29.745129,-95.49131,0.828460,0.619048,0.040000
16,Briargrove Park/Walnut Bend,77042,0.132543,29.741565,-95.55996,0.807018,0.485714,0.146667
17,Briarmeadow/Tanglewilde,77063,0.072198,29.734379,-95.52269,0.771930,0.673469,0.293333
22,Charnwood/Briarbend,77063,0.156789,29.734379,-95.52269,0.771930,0.673469,0.293333
37,Denver Harbor,77020,0.005029,29.775927,-95.31836,0.526316,0.532468,0.146667
42,Energy Corridor,77077,0.087644,29.750897,-95.61255,0.851852,0.510204,0.040000
52,Gulfton,77081,0.063398,29.708280,-95.48361,0.697856,0.693878,0.226667
57,Hobby Area,77061,0.012033,29.660280,-95.28446,0.703704,0.542857,0.120000
82,Medical Center South,77051,0.011135,29.665430,-95.36871,0.530214,0.142857,0.040000


When we examine cluster 3, we find that in this cluster, the home prices are on relatively reasonable side, Crime rate is average. School ratings are high except Upper Kirby. These neighborhoods have many venues for different activities. 

We would  like to recommend all nighborhoods in Zip Code -77007


#### Cluster 4

In [67]:
nbrs_final_clustered.loc[nbrs_final_clustered['Cluster Labels'] == 3, nbrs_final_clustered.columns[[1] + list(range(2, nbrs_final_clustered.shape[1]))]]

,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating,Venues
83,Memorial Close In,77024,0.810776,29.773994,-95.51771,0.368421,0.959184,0.000000
108,River Oaks Area,77019,0.755388,29.752528,-95.39923,0.401559,0.714286,0.346667
109,Rivercrest,77042,1.000000,29.741565,-95.55996,0.807018,0.485714,0.146667
129,Tanglewood Area,77056,0.557651,29.747328,-95.46931,0.637427,0.809524,0.186667


Neighborhoods in this cluster have highest home prices. Public school ratings are not the best except 77024. Also number of venues in the neighborhoods are not that many. 

We would recommend Zip Code - 77024 for affluent families who are okay with little less venues for activities in their own neighborhood.

#### Cluster 5

In [68]:
nbrs_final_clustered.loc[nbrs_final_clustered['Cluster Labels'] == 4, nbrs_final_clustered.columns[[1] + list(range(2, nbrs_final_clustered.shape[1]))]]

,Neighborhood,Zip,HomePrice,Latitude,Longitude,Crimes,Rating,Venues
30,Cottage Grove,77007,0.108297,29.772627,-95.40319,0.803119,1.000000,0.613333
84,Memorial Park,77007,0.373204,29.772627,-95.40319,0.803119,1.000000,0.613333
106,Rice Military/Washington Corridor,77007,0.128915,29.772627,-95.40319,0.803119,1.000000,0.613333
137,Upper Kirby,77098,0.258064,29.735529,-95.41405,0.360624,0.657143,1.000000
139,Washington East/Sabine,77007,0.114224,29.772627,-95.40319,0.803119,1.000000,0.613333


##### When we look into the cluster 5, we find that in this cluster the Home Price is on lower side, Crime rate is average to high .  School Rating are average.  Number of Venues in neighborhoods are also lower. 